In [2]:
# 15차시기준
# 국회입법조사처 연구보고서
# https://www.nars.go.kr/report/list.do?page=1&cmsCode=CM0043&categoryId=&searchType=&searchKeyword=

# '#content > div.ls-box > ul > li:nth-child(1)'

#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt

import random
import os   # 폴더 생성하기 위해 필요함

In [3]:
#Step 2. 사용자에게 크롤링 정보를 입력 받습니다.
print("=" *80)
print("서울시 응답소 게시판 크롤링하기")
print("=" *80)

query_txt = '서울시응답소'


cnt = int(input('1.크롤링 할 건수는 몇건입니까?: '))
page_cnt = math.ceil(cnt / 10)

f_dir = input("2.결과 파일을 저장할 폴더명만 쓰세요(예:c:\\temp\\):")

# 저장될 파일위치와 이름을 지정합니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'

서울시 응답소 게시판 크롤링하기
1.크롤링 할 건수는 몇건입니까?: 30
2.결과 파일을 저장할 폴더명만 쓰세요(예:c:\temp\):c:\doit\


In [8]:
#Step 3. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.

s_time = time.time( )

path = "c:/doit/chromedriver_win32/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://eungdapso.seoul.go.kr/')

# time.sleep(random.randrange(2,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택

time.sleep(5)

#Step 4. 여러 메뉴 중에서 원순씨에게 바랍니다 게시판 선택
driver.find_element_by_xpath("""//*[@id="gnb"]/div[4]/div[2]/div/a""").click()

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=85.0.4183.102)


In [10]:
#시간을 계산하기 위해 현재 타임스탬프 찍기
now1 = time.localtime()
s1 = '%04d-%02d-%02d-%02d:%02d:%02d' % (now1.tm_year, now1.tm_mon, now1.tm_mday, now1.tm_hour, now1.tm_min, now1.tm_sec)

print("\n")
page_cnt = math.ceil(cnt / 10)
print("크롤링 할 총 페이지 번호: ",page_cnt)
print("=" *80)

# 소주제 1: 전체 요약 정보 추출하여 출력하기

# Step 4.  게시글 요약 정보 출력

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

result_list= soup.find('div', class_='pclist_table mt20').get_text( )
print(result_list)



크롤링 할 총 페이지 번호:  3


번호
제목
신청일
조회수

 

11784

 서울특별시 교통지도과에 권한을 좀 더...

2019-01-15
47


 

11783

 서울시청 광장활용 방안제시

2019-01-14
19


 

11782

 남산공원 샤워실 이용을 허하여 주십시...

2019-01-13
24


 

11781

 참고하겠다는 진정성도 부족하고 말입니...

2019-01-12
23


 

11780

 접수번호 20181213903152 ...

2019-01-09
15


 

11779

 창동역일대를번화가개발과아레나공연장건립...

2019-01-09
16


 

11778

 보라매독서실  예산   지원하라!!!

2019-01-09
56


 

11777

 서울시 심사통과된 시설계획 이행바랍니...

2019-01-09
33


 

11776

 2019 서울시 시무식에 대해 시장님...

2019-01-09
27


 

11775

 보라매 공원 독서실 이용시간 단축과 ...

2019-01-09
23





In [11]:
# 소주제 2: 현재 창에서 열리는 페이지의 내용을 추출 한 후 돌아가기로 이전 페이지 리턴하기

# Step 5. 현재 페이지의 내용을 저장 목록을 만든 후 목록에 있는 내용을 파일에 저장하기

no2=[]           # 게시글 번호 컬럼
contents=[]        # 게시글 내용 컬럼

no = 1

click_cnt = 1    # 현재 페이지 번호

for x in range(1,page_cnt+1) :
    print("%s 페이지 내용 수집 시작합니다 =======================" %x)
        
    # 1 페이지에 있는 게시글 10건의 상세 내역 가져오기
    for i in range(1,11) :
        driver.find_element_by_xpath("""//*[@id="content_cont"]/div[2]/div/form/div[4]/div[%s]/ul/li[2]/a"""  %i).click( )
                               
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        content = soup.find('div','table_inner_desc').get_text()
        print(content)
   
        no2.append(no)                             # 게시글 번호에 추가
        contents.append(content.replace("\n","")+'\n')   # 내용에 추가
       
        no += 1
        
        time.sleep(1)
        
        driver.back()
        
    time.sleep(2)        # 페이지 변경 전 2초 대기 

    if click_cnt > page_cnt+1 :
        break
        
    click_cnt += 1
    
    if (click_cnt % 5 == 0):
        driver.find_element_by_link_text('''다음''').click()
    else :
        driver.find_element_by_link_text("""%s""" %click_cnt).click() # 다음 페이지번호 클릭

1 페이지 내용 수집 시작합니다 =======================

Q. 상담내용
									안녕하세요. 저는 종로구 창신동에 사는 000이라고 합니다.창신동은 잘 아시겠지만 좁은 골목과 높은 언덕으로 유명한 종로의 대표적인 달동네입니다.만약 불이라도 나면 정말 엄청난 재난이 발생할 수 있는 곳이지요.또한 주차공간이 협소하여 불법주차가 많이 발생합니다.따라서 구청의 현장 단속만으로는 한계가 있어 신고앱을 통한 과태료 부과가 큰 재난을 막고 행정 낭비를 최소화 할 수 있다고 생각합니다.그런데 이번에 제가 신고한 소화전 불법주차의 경우 과태료부과를 할 수가 없다고 합니다(첨부파일 사진 참조 부탁드리며 번호판은 만약을 위해 가렸으나 원본은 그대로 표시되어 있습니다)그 이유를 교통지도과의 김유빈님이라는 분으로부터 들었습니다.- 18년 8월에 법이 바뀌어서 아직 시민들 인식이 부족하다- 맨홀식 소화전은 눈에 잘 안 띄어 디자인이 변경되면 그 때 단속하겠다- 제가 신고한 건은 그나마 인식이 가능하지만 다른 여타 신고건의 경우 맨홀식 소화전 인식이 힘든 경우도 있어 &quot;일괄적&quot;으로 맨홀식 소화전은 과태료 부과 처리를 하지 않고 있다.저것들이 큰 재앙들을 불러올 수 있는 행위에 대해 사진만으로 과태료를 부과할 수 없는 이유랍니다. 서울시에서는 하지 못하니 각 구청에 전화해서 현장단속으로 요청하라고 합니다. 그러면 신고앱은 도대체 뭐하러 만든겁니까? 저처럼 신고하느라, 민원 확인하느라, 이유 물어보느라 시민들 시간 낭비하라고 만든건가요? 누가봐도 명백히 소화전 위에 주차한 것이 확인이 가능한 제 사진도 정말 이해가 가지 않는 이유들로 결국 과태료 부과를 못하겠다고 합니다.제 생각엔 서울시 교통지도과가 힘이 정말 하나도 없는 것 같습니다. - 계도기간(방송에서 지겹도록 소방법이 바뀐다고 알려졌습니다. 제천 사건 때문이지요)도 충분히 있었고 심지어 법 자체가 바뀐지 반년이 지나고 있습니다. 그런데 시민들 인식이 미흡하다구요? 대체 어디 시민이 아직도


Q. 상담내용
									 저는 서울시민이자 두 아이의 아빠입니다 얼마 전까지 저는 동부도로사업소내에 교통안전시설경정비팀에 근무를 하다 12월31일부로 계약만료로 인한 퇴직을 하였습니다, 소속은 도로교통공단소속으로 신호기정비용역을 서울시에서 도로교통공단과의 계약이 원활히 이루어지지 않았으며 수의계약에서 공개입찰방식으로 변경되어 2회유찰된 상황에 공단에서의 계약만료로 인해 해고통보를 받았습니다. 아시다시피 하루하루 커가는 아이들에게는 금전적인 요소가 많이 있습니다.이러한 상황에 급작스런 해고통보로 인한 저희 가정은 사실상 생계유지가 난처한 현 상황에 이르렀습니다. 또한 1월부터는 저의 실직으로 인해 소득이 전혀 없는 상태이며 가장으로서 두 아이들에게는 더더욱 미안한 감정만 들며 현재 눈앞이 캄캄합니다. 지금 이 상황은 저 혼자만의 일이 아닌 제 동료직원이었던 분들과 직원들의 가정에도 직결 되는 문제입니다. 지난해 2018년도에 서울시교육청에서는 용역근로자 4000여명 직접 고용한다는 내용을 접했습니다. 공공기관 비정규직은 그동안 정부의 가이드라인에 따라 정규직 전환을 추진해 온걸로 알고 있습니다. 2019년 1월을 맞이했으나 신호기 정비용역은 정비업체선정이 안된 것은 물론이거니와 1월8일인 오늘까지도 입찰공지까지 올라오지 않은 상태입니다. 이러한 상황으로 봤을 때 신호기 정비를 했던 직원들이 다시 한번 서울시 용역근로자가 아닌 서울시 소속근무자로서 신호기 정비 업무를 수행할 수 있도록 시장님께 요청드리는 바입니다.


Q. 상담내용
									한통의 메일이 오늘을 밝게 맞이한다박인숙님과 같은 공무원이 있기에 서울시민은 행복합니다바쁘신중에도 당신이 있기에 서울시는 세계속에 자리메김  될거라 생각됩니다다시한번 감사드리면서2019년도에는 가정에 건강과 행복이 가하시기 기도 드립니다아울러 뜻하시는 바램 소원 성취바랍니다


Q. 상담내용
									저희 7살 쌍둥이 아이들이 또래에 비해 언어가 지연되고있습니다.그래서 어린이집장애통합반에 들어가려고 하니


Q. 상담내용
									신분당선 서북부연장의 일부 노선과 관련해서 궁금한 점이 있어 글을 남깁니다.물론 예타 결과가 나와봐야 알 수 있겠지만 국토교통부 초기 계획안에서는 세검정초교역이었던 것이 시간이 지나면서 어느 순간부터 상명대역으로 불리기 시작했습니다. 지역주민으로서 상명대학교의 로비가 있지 않았을까? 하는 의구심을 갖게 하는 부분입니다.지역주민이라면 모두 잘 아시겠지만 세검정 삼거리(상명대 입구)에 역이 생기면 이는 상명대학교만을 위한 역이 되는 것입니다. 최초 계획대로 신영동 삼거리(세검정초교)에 역이 생겨야 구기동, 평창동, 신영동, 홍지동, 상명대, 부암동 일부, 북한산을 찾는 등산객들이 모두 이용할 수 있습니다. 물론 아직 아무것도 결정된 것이 없지만 만약 예타 결과가 통과된다면 상명대학교만을 위한 역이 생기는 것은 절대 막아야 한다고 생각합니다.
								

Q. 상담내용
									박원순 서울시장님이번에 오랫만에 경기도로 이사를 간 친구를 만났습니다. 그 친구는 6개월 전까지 저랑 같이 서울에서 2년 정도 택시를 하던 친구였으며,점심식사를 하면서 최근에  핫 이슈인 카카오 카풀등의 이야기를 하던중, 경기도는 택시기사들에게 월 10만원씩 지원해 준다고 이야기를 하며. 올해부터 시행되었고 이번 추석때 1인당 7월, 8월, 9월분 한번에 30만원이라는 돈을 지급해 주었으며 지금은 매달 10만원씩 지원하여 준다고 자랑을 하더군요.경기도에서 월 10만원씩 직접 지원을 받기위해서는첫째로 가해교통사고가 없는 택시기사. 두번째로 승차거부나 불친절로 민원이 접수되어 처벌을 받지 않은 택시기사이 두가지 조건이 있고..이 친구는 월 10만원을 받기 위해 항상 승/하차시 친절하게 인사하고  항상 얼굴로 대하며, 운행시 교통사고가 나지 않게 조금 더 조심하게 된다고 합니다. 물론 승차거부는 상상도 못한다고 합니다.이런 작은 복지로 인해 이재명 경기도지사님은 경기도 택시기사들에게는 인지도가 엄청나게 급상승 하였습니다서울은 어떻습니까?요금도 오른다고 

In [12]:
seoul = pd.DataFrame()
seoul['번호']=no2
seoul['내용']=contents

# txt 형식의 파일로 저장하기
f = open(ff_name, 'a',encoding='UTF-8')     
f.write(str(contents)+'\n')
f.close( )
        
# csv 형태로 저장하기
seoul.to_csv(fc_name,encoding="utf-8-sig")

# 엑셀 형태로 저장하기
seoul.to_excel(fx_name)

e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

print("\n") 
print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

driver.close( )



총 소요시간은 837.4 초 입니다 
파일 저장 완료: txt 파일명 : c:\data\2019-01-18-15-43-10-서울시응답소\2019-01-18-15-43-10-서울시응답소.txt 
파일 저장 완료: csv 파일명 : c:\data\2019-01-18-15-43-10-서울시응답소\2019-01-18-15-43-10-서울시응답소.csv 
파일 저장 완료: xls 파일명 : c:\data\2019-01-18-15-43-10-서울시응답소\2019-01-18-15-43-10-서울시응답소.xls 
